In [1]:
import numpy as np
import pandas as pd
import torch
from torchvision import transforms, models, datasets
from waggle.plugin import Plugin
from waggle.data.vision import Camera

In [2]:
device =torch.device("cuda" if torch.cuda.is_available() else "cpu")
device
CUDA_LAUNCH_BLOCKING=1

In [28]:
root = "./images"
transformation = transforms.Compose([transforms.Resize((224,224)),
                                    transforms.ToTensor()])
data = datasets.ImageFolder(root = root, transform=transformation)

test_size = int(0.2 * len(data))
train_size = len(data) - test_size
train_set, test_set = torch.utils.data.random_split(data, [train_size, test_size])



219
876


In [9]:
#Create additional layers for snow classification
model = models.resnet50(pretrained = True)
for param in model.parameters():
    param.requires_grad = False   

model.fc = torch.nn.Sequential(
    torch.nn.Linear(2048, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256,2),
    torch.nn.Softmax(dim= 1)
)
model.to(device)

c:\Users\alxto\Anaconda3\envs\sage\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\alxto\Anaconda3\envs\sage\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [16]:
#parameters

lr = 0.0003
batch_size = 32
epochs =100
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.fc.parameters(), lr)
train_loader = torch.utils.data.DataLoader(train_set, batch_size= batch_size, shuffle= True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size= batch_size, shuffle= False)

In [29]:

#training loop
running_loss = 0 
train_loss, test_loss = [],[]
model.train()
verbose = 1
steps = 0
for epoch in range(epochs):
    print(epoch)
    for images,labels in train_loader:
        steps +=1
        print(steps)
        labels = torch.nn.functional.one_hot(labels)
        images,labels = images.to(device), labels.to(device).float()
        optimizer.zero_grad()
        out = model(images)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if steps % verbose == 0:
            print('here?')
            val_loss = 0
            num_correct = 0
            model.eval()
            with torch.no_grad():
                
                for i, l in test_loader:
                    l=torch.nn.functional.one_hot(l)
                    i,l = i.to(device), l.to(device).float()
                    t_out = model(i)
                    tb_loss = criterion(t_out, l)
                    val_loss += tb_loss.item()
                    t_out = np.argmax(t_out.cpu().numpy(), axis = 1)
                    l = np.argmax(l.cpu().numpy(), axis = 1)
                    num_correct += np.sum(t_out ==l)/2

                    
                    
                train_loss.append(running_loss/len(train_loader))
                test_loss.append(val_loss/len(test_loader))
                print(f"train: {running_loss/len(train_loader)}")
                print(f"test: {val_loss/len(test_loader)}")
                print(f'test_accuracy: {num_correct/len(test_set)}')
            running_loss= 0
            model.train()
    torch.save(model, 'model.pth')


        
    

0
1
here?
7
